In [1]:
from random import choice, choices
import base64
from Crypto.Cipher import AES

BLOCKSIZE = 16

In [12]:
def XOR(A: bytes, B: bytes):
    assert type(A) is bytes and type(B) is bytes and len(A) == len(B)
    return bytes([ a^b for a, b in zip(A, B) ])

def IntToLittleEndian(x: int, nbytes: int):
    assert type(x) is int and x >= 0
    out = bytes()
    while x >  0:
        out += bytes([ x & 0xFF ])
        x >>= 8
    
    assert len(out) <= nbytes
    out += bytes( [0] * ( nbytes-len(out) ) )
    
    return out

def LittleEndianToInt(x: bytes):
    assert type(x) is bytes
    out = 0
    byte_counter = 0
    while len(x) > 0:
        out += x[0] << byte_counter*8
        byte_counter += 1
        x = x[1:]
        
    return out

In [8]:
little_endian = IntToLittleEndian(24, 8)
print(little_endian)

b'\x18\x00\x00\x00\x00\x00\x00\x00'


In [9]:
LittleEndianToInt(little_endian)

24

In [46]:
class CTR(object):
    def __init__(self, nonce: bytes, key: bytes):
        assert type(nonce) is bytes and len(nonce) < BLOCKSIZE
        self.nonce = nonce
        self.aes = AES.new(key, AES.MODE_ECB)
        
    def EncryptBlock(self, block: bytes, counter: int):
        assert type(counter) is int and counter >= 0
        assert type(block) is bytes and len(block) == BLOCKSIZE
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block, block)
    
    def DecryptBlock(self, block, counter: int):
        return self.EncryptBlock(block, counter)
    
    def EncryptBytes(self, plaintext: bytes, counter: int):
        assert type(plaintext) is bytes and len(plaintext) <= BLOCKSIZE
        assert type(counter) is int and counter >= 0
        little_endian: bytes = IntToLittleEndian(counter, nbytes = BLOCKSIZE-len(self.nonce))
        key_block = self.aes.encrypt( self.nonce + little_endian )
        assert type(key_block) is bytes and len(key_block) == BLOCKSIZE
        return XOR(key_block[:len(plaintext)], plaintext)
    
    def DecryptBytes(self, ciphertext: bytes, counter: int):
        return self.EncryptBytes(ciphertext, counter)
    
    def EncryptStream(self, plaintext: bytes, counter: int = 0):
        assert type(plaintext) is bytes
        stream = bytes(plaintext)
        ciphertexts = list()
        
        while len(stream) > 0:
            block =  stream[:BLOCKSIZE]
            stream = stream[BLOCKSIZE:]
            assert ( len(block) == BLOCKSIZE ) ^ ( len(stream) == 0 )
            ciphertexts += [ self.EncryptBytes(block, counter) ]
            counter += 1
        
        ciphertext = b''.join(ciphertexts)
        assert len(ciphertext) == len(plaintext)
        return ciphertext
    
    def DecryptStream(self, plaintext: bytes, counter: int = 0):
        return self.EncryptStream(plaintext, counter)
    
        

In [47]:
ctr_cipher = CTR(nonce = b'\0'*8, key = b'YELLOW SUBMARINE')
plaintext = b'Cooking MCs like a pound of bacon'
ciphertext = ctr_cipher.EncryptStream(plaintext, 0)
print(ciphertext)

b'5\xbe\xa4 \xc6\xcc!\xc2\xae\xecp}\x00z\xa8\x17\xf2\x8dL\xac\xf7\x18|\xba\xef\xb5y\xb3\xcd\x8f\x10wC'


In [49]:
ctr_cipher.DecryptStream(ciphertext, 0)

b'Cooking MCs like a pound of bacon'